# Imports and loading data

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import os

Using TensorFlow backend.


**Reading Data**

In [ ]:
sem1A = pd.read_csv('/content/drive/My Drive/CSV/Final/2017_Sem1_A_batch.csv',error_bad_lines=False, skiprows = 1)
sem1B = pd.read_csv('/content/drive/My Drive/CSV/Final/2017_Sem1Final_Bgroup.csv',error_bad_lines=False)
sem2A = pd.read_csv('/content/drive/My Drive/CSV/Final/_2017_sem2_A - 2017_sem2.csv',error_bad_lines=False, skiprows=1)
sem2B = pd.read_csv('/content/drive/My Drive/CSV/Final/2017_sem2_Bgroup.csv - 2017_sem2.csv',error_bad_lines=False)

## Batch A Semester 1

In [ ]:
sem1A.head()

,Sr.No. Name,Roll No.\nCredits,MA101\n4,AP101\n4,AC101\n4,ME101\n4,ME103\n2,HU101\n3,TC\n21,SGPA,Unnamed: 10
0,1 SANGRAM TYAGI,2017/A1/01,O,O,O,A,B+,B,21,8.76,NaN
1,2 JAGNUR SINGH SANDHU,2017/A1/02,O,O,O,A,A,A,21,9.14,NaN
2,3 ANIL BANSAL,2017/A1/03,O,A+,O,A,A,B+,21,8.81,NaN
3,4 SHRISTI DABOLA,2017/A1/04,A+,A+,A,B+,B+,A,21,8.1,NaN
4,5 ABHIRAJ TULSYAN,2017/A1/05,O,O,A+,A,B+,B+,21,8.71,NaN


In [ ]:
def modify(df,new_names, cols2drop):  # changing column names
  df = df.drop(cols2drop,axis =1 )
  dicto = dict(zip(list(df.columns),new_names))
  df = df.rename(columns =dicto)
  return df

In [ ]:
def remove_useless(df): # removing students who did not attend exams (please attend exams)
  for i in df.columns[2:-1]:
    for index,row in df.iterrows():
      if (row[i] != 'O' and row[i] != 'A+' and row[i] != 'A' and row[i] != 'B+' and row[i] != 'B' and row[i] != 'C' and row[i] != 'P' and row[i] != 'F' and row[i] != 'D' and row[i] != 'D+' and row[i] != 'C+'):
        df = df.drop(index = int(index))
  return df 

In [ ]:
new_name=['Name', 'Roll No.', 'Math1', 'Physics1', 'Chem', 'BME','Workshop','FEC1','SGPGA1']
todrop = ['TC\n21','Unnamed: 10']
sem1A = modify(sem1A,new_name,todrop)
sem1A = remove_useless(sem1A)
sem1A.head() # our semester 1 A batch data 

,Name,Roll No.,Math1,Physics1,Chem,BME,Workshop,FEC1,SGPGA1
0,1 SANGRAM TYAGI,2017/A1/01,O,O,O,A,B+,B,8.76
1,2 JAGNUR SINGH SANDHU,2017/A1/02,O,O,O,A,A,A,9.14
2,3 ANIL BANSAL,2017/A1/03,O,A+,O,A,A,B+,8.81
3,4 SHRISTI DABOLA,2017/A1/04,A+,A+,A,B+,B+,A,8.1
4,5 ABHIRAJ TULSYAN,2017/A1/05,O,O,A+,A,B+,B+,8.71


## Batch A Semester 2

In [ ]:
new_name=['Name', 'Roll No.', 'Math2', 'Physics2', 'EE', 'CO','ED','FEC2','SGPGA2']
todrop = ['TC','Unnamed: 10']
sem2A = modify(sem2A,new_name,todrop)
sem2A = remove_useless(sem2A)
sem2A.head() # our semester 2 batch a data

,Name,Roll No.,Math2,Physics2,EE,CO,ED,FEC2,SGPGA2
1,1 SANGRAM TYAGI,2017/A1/01,A+,B+,B+,A+,F,B,6.95
2,2 JAGNUR SINGH SANDHU,2017/A1/02,O,O,A,A+,B,A+,8.9
3,3 ANIL BANSAL,2017/A1/03,O,O,A+,A,B,A,8.76
4,4 SHRISTI DABOLA,2017/A1/04,O,A+,A+,A+,B,A+,8.9
5,5 ABHIRAJ TULSYAN,2017/A1/05,O,O,A,O,P,B+,8.62


## Merging semester 1 and semester 2


In [ ]:
def merge_sem(df1,df2):
  list1 = list(df1['Roll No.'])
  list2 = list(df2['Roll No.'])
  list3 = [value for value in list1 if value in list2]
  bl1 = [i in list3 for i in df1['Roll No.']]
  bl2 = [i in list3 for i in df2['Roll No.']]
  new_sem1 = df1[bl1]
  new_sem2 = df2[bl2]
  new_sem1 = new_sem1.reset_index()
  new_sem2 = new_sem2.reset_index()
  new = pd.concat([new_sem1, new_sem2], axis=1)
  new = new.loc[:,~new.columns.duplicated()]
  new =new.drop(['index'], axis=1)
  return new

In [ ]:
batch_A_results = merge_sem(sem1A, sem2A)
batch_A_results.head()b # semester 1 and 2 a batch combined 

,Name,Roll No.,Math1,Physics1,Chem,BME,Workshop,FEC1,SGPGA1,Math2,Physics2,EE,CO,ED,FEC2,SGPGA2
0,1 SANGRAM TYAGI,2017/A1/01,O,O,O,A,B+,B,8.76,A+,B+,B+,A+,F,B,6.95
1,2 JAGNUR SINGH SANDHU,2017/A1/02,O,O,O,A,A,A,9.14,O,O,A,A+,B,A+,8.9
2,3 ANIL BANSAL,2017/A1/03,O,A+,O,A,A,B+,8.81,O,O,A+,A,B,A,8.76
3,4 SHRISTI DABOLA,2017/A1/04,A+,A+,A,B+,B+,A,8.1,O,A+,A+,A+,B,A+,8.9
4,5 ABHIRAJ TULSYAN,2017/A1/05,O,O,A+,A,B+,B+,8.71,O,O,A,O,P,B+,8.62


### Reordering batch A 

In [ ]:
recol=['Name', 'Roll No.', 'Math1', 'Physics1', 'EE', 'CO', 'ED', 'FEC1',
       'SGPGA1', 'Math2', 'Physics2', 'Chem', 'BME', 'Workshop', 'FEC2',
       'SGPGA2']
batch_A_results = batch_A_results[recol]
batch_A_results.head() 

,Name,Roll No.,Math1,Physics1,EE,CO,ED,FEC1,SGPGA1,Math2,Physics2,Chem,BME,Workshop,FEC2,SGPGA2
0,1 SANGRAM TYAGI,2017/A1/01,O,O,B+,A+,F,B,8.76,A+,B+,O,A,B+,B,6.95
1,2 JAGNUR SINGH SANDHU,2017/A1/02,O,O,A,A+,B,A,9.14,O,O,O,A,A,A+,8.9
2,3 ANIL BANSAL,2017/A1/03,O,A+,A+,A,B,B+,8.81,O,O,O,A,A,A,8.76
3,4 SHRISTI DABOLA,2017/A1/04,A+,A+,A+,A+,B,A,8.1,O,A+,A,B+,B+,A+,8.9
4,5 ABHIRAJ TULSYAN,2017/A1/05,O,O,A,O,P,B+,8.71,O,O,A+,A,B+,B+,8.62


## Batch B Semester 1

In [ ]:
new_name=['Name', 'Roll No.', 'Math1', 'Physics1', 'EE', 'CO','ED','FEC1','SGPGA1']
todrop = ['TC','Unnamed: 10']
sem1B = modify(sem1B,new_name,todrop)
sem1B = remove_useless(sem1B)
sem1B.head() # # our semester 1 batch B data

,Name,Roll No.,Math1,Physics1,EE,CO,ED,FEC1,SGPGA1
0,1 VASU GUPTA,2017/B1/67,O,A+,O,O,A+,A,9.43
1,2 TANISH GROVER,2017/B1/68,O,O,O,O,A,B,9.24
2,3 DHANANJAY NEGI,2017/B1/69,A+,A,B,A+,B+,B,7.62
3,4 AMIT KUMAR SINGH,2017/B1/70,A,O,O,A,B+,B,8.38
4,5 RITIK AGGARWAL,2017/B1/71,A+,O,O,A,A,B,8.67


## Batch B Semester 2

In [ ]:
new_name=['Name', 'Roll No.', 'Math2', 'Physics2', 'Chem', 'BME','Workshop','FEC2','SGPGA2']
todrop = ['TC\n21','Unnamed: 10']
sem2B = modify(sem2B,new_name,todrop)
sem2B = remove_useless(sem2B)
sem2B.head()# our semester 2 batch B data

,Name,Roll No.,Math2,Physics2,Chem,BME,Workshop,FEC2,SGPGA2
0,1 VASU GUPTA,2017/B1/67,O,O,O,O,O,A,9.71
1,2 TANISH GROVER,2017/B1/68,A+,O,O,A+,B+,A+,9.19
2,3 DHANANJAY NEGI,2017/B1/69,B+,A+,A,B+,A+,B+,7.76
3,4 AMIT KUMAR SINGH,2017/B1/70,O,O,O,O,O,A,9.71
4,5 RITIK AGGARWAL,2017/B1/71,O,A+,O,O,A+,A,9.43


In [ ]:
batch_B_results=merge_sem(sem1B,sem2B)
temp = merge_sem(sem1B.tail(10), sem2B.tail(10))
temp=pd.concat([batch_B_results, temp])
temp.drop_duplicates(['Roll No.'], keep = 'last', inplace = True)
batch_B_results = temp.reset_index() # merging sem1 b and sem 2 b students data

___
___
___

In [ ]:
Final = pd.concat([batch_B_results,batch_A_results])
Final.reset_index()
Final.drop(['index'], axis = 1)
Final.head() # our B batch data

,index,Name,Roll No.,Math1,Physics1,EE,CO,ED,FEC1,SGPGA1,Math2,Physics2,Chem,BME,Workshop,FEC2,SGPGA2
0,0.0,1 VASU GUPTA,2017/B1/67,O,A+,O,O,A+,A,9.43,O,O,O,O,O,A,9.71
1,1.0,2 TANISH GROVER,2017/B1/68,O,O,O,O,A,B,9.24,A+,O,O,A+,B+,A+,9.19
2,2.0,3 DHANANJAY NEGI,2017/B1/69,A+,A,B,A+,B+,B,7.62,B+,A+,A,B+,A+,B+,7.76
3,3.0,4 AMIT KUMAR SINGH,2017/B1/70,A,O,O,A,B+,B,8.38,O,O,O,O,O,A,9.71
4,4.0,5 RITIK AGGARWAL,2017/B1/71,A+,O,O,A,A,B,8.67,O,A+,O,O,A+,A,9.43


In [ ]:
temp = ['Math1', 'Physics1', 'EE', 'CO', 'ED', 'FEC1', 'Math2', 'Physics2', 'Chem', 'BME', 'Workshop', 'FEC2',]

In [ ]:
grade2idx = {'F':0,'P':1,'C':2,'B':3,'B+':4,'A':5,'A+':6,'O':7}
for col in temp:
  Final[col]=pd.Series([grade2idx[i] for i in Final[col]])
Final.head()  # converting grades to numerical based grading using grade2idx dictionary

,index,Name,Roll No.,Math1,Physics1,EE,CO,ED,FEC1,SGPGA1,Math2,Physics2,Chem,BME,Workshop,FEC2,SGPGA2
0,0.0,1 VASU GUPTA,2017/B1/67,7,6,7,7,6,5,9.43,7,7,7,7,7,5,9.71
1,1.0,2 TANISH GROVER,2017/B1/68,7,7,7,7,5,3,9.24,6,7,7,6,4,6,9.19
2,2.0,3 DHANANJAY NEGI,2017/B1/69,6,5,3,6,4,3,7.62,4,6,5,4,6,4,7.76
3,3.0,4 AMIT KUMAR SINGH,2017/B1/70,5,7,7,5,4,3,8.38,7,7,7,7,7,5,9.71
4,4.0,5 RITIK AGGARWAL,2017/B1/71,6,7,7,5,5,3,8.67,7,6,7,7,6,5,9.43


In [ ]:
CGPA1=(Final['Math1']+3)+(Final['Physics1']+3)+(Final['EE']+3)+(Final['CO']+3)+(Final['ED']+3)/2+(Final['FEC1']+3)/2
CGPA1=CGPA1/5
Final['SGPGA1']=CGPA1
CGPA2=(Final['Math2']+3)+(Final['Physics2']+3)+(Final['Chem']+3)+(Final['BME']+3)+(Final['Workshop']+3)/2+(Final['FEC2']+3)/2
CGPA2=CGPA2/5
Final['SGPGA2']=CGPA2
Final.head()# final data after combining everything 

,index,Name,Roll No.,Math1,Physics1,EE,CO,ED,FEC1,SGPGA1,Math2,Physics2,Chem,BME,Workshop,FEC2,SGPGA2
0,0.0,1 VASU GUPTA,2017/B1/67,7,6,7,7,6,5,9.5,7,7,7,7,7,5,9.8
1,1.0,2 TANISH GROVER,2017/B1/68,7,7,7,7,5,3,9.4,6,7,7,6,4,6,9.2
2,2.0,3 DHANANJAY NEGI,2017/B1/69,6,5,3,6,4,3,7.7,4,6,5,4,6,4,7.8
3,3.0,4 AMIT KUMAR SINGH,2017/B1/70,5,7,7,5,4,3,8.5,7,7,7,7,7,5,9.8
4,4.0,5 RITIK AGGARWAL,2017/B1/71,6,7,7,5,5,3,8.8,7,6,7,7,6,5,9.5


## Saving the data

In [ ]:
Final.to_csv('/content/final.csv',index = False) # converting everything to csv file for use

In [ ]:
dfp = pd.read_csv('/content/16Fi.csv' )
dfp.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace = True)
dfp.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace = True)
dfp['Math1']-=3
dfp['Physics1']-=3
dfp['EE']-=3
dfp['CO']-=3
dfp['ED']-=3
dfp['FEC1']-=3
dfp['Math2']-=3
dfp['Physics2']-=3
dfp['Chem']-=3
dfp['BME']-=3
dfp['Workshop']-=3
dfp['FEC2']-=3
dfp.head()
vp=pd.concat([Final,dfp])
vp.reset_index()
vp.head()
vp.to_csv('/content/vp.csv',index = False)